In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 
import time
import collections as co

In [ ]:
def get_observer(chrome_driver = "/Users/USMAC/Desktop/chromedriver"):
    driver = webdriver.Chrome(chrome_driver)
    driver.get("http://www.tsetmc.com/Loader.aspx?ParTree=15131F")
    content = driver.page_source
    soup = BeautifulSoup(content)
    rows = soup.findAll('div', attrs={'class':'{c}'})
    observer_table = {}

    for row in rows:
        factors = {'symbol':'', "name":'', "num_shares":'', "amount":'', "value":'', "value_yesterday":'',
               "value_first":"", "last_deal_price":'', "last_deal_fluctuation":'',
               "last_deal_fluctuation_percentage":'', "last_price_price":"", "last_price_fluctuation":'',
               "last_price_fluctuation_percentage":'', "lowest_price":'', "highest_price":'', "eps":'', "pe":'',
               "buy_number_of_shares":'', "buy_amount":'', "buy_price":'', "sell_price":'', "sell_amount":'',
               "sell_number_of_shares":''}
        for i,ele in enumerate(row.stripped_strings):
            factors[list(factors.keys())[i]] = ele

        factors['link_id'] = row.attrs['id']

        observer_table[factors['symbol']] = factors
        
    
    driver.quit()
    return observer_table


observer_table = get_observer()

In [ ]:
def get_history(content):
   

    rows_ev = []
    rows_odd = []
    
    while len(rows_ev)==0 or len(rows_odd)==0:
        soup = BeautifulSoup(content)
        rows_ev = soup.findAll('tr', attrs={'class':'ev_modern '})
        rows_odd = soup.findAll('tr', attrs={'class':'odd_modern '})
        
    rows = rows_ev + rows_odd
    history_table = {}
        
    for row in rows:
        
        factors = {"link_id":"", "max_price":"", "min_price":"", "last_price_fluctuation_percentage":'', "last_price_fluctuation":'',
                   "last_price_price":"", "last_deal_fluctuation_percentage":'', "last_deal_fluctuation":'', "last_deal_price":'',
                   "value_first":"", "value_yesterday":'', "value":'', "amount":'', "num_shares":'', "date":""}
        
        tmp = []
        for i,ele in enumerate(row.stripped_strings):
            if not ((str(ele).find("M") != -1) or (str(ele).find("B") != -1)): 
                if not ((len(tmp) != 0) and (tmp[-1] == ele.replace(",", "")) and (ele.find(",") > -1)):
                    tmp.append(ele)                
               

        
        try:
            factors = {list(factors.keys())[i]:ele for i, ele in enumerate(tmp)}
            history_table[factors['date']] = factors
        
        except:
            print("the record on {} is not readable!".format(tmp[-1]))
            #print("*****************************")
            #print(tmp)
            #print(len(tmp))
            #print(factors.keys())
            #print(len(factors.keys()))
            #print(factors)
            
    return history_table

        


In [ ]:
def get_all_history(share_id, chrome_driver):


    driver = webdriver.Chrome(chrome_driver)
    
    item = None
    while item is None:
        try:
            driver.get("http://www.tsetmc.com/loader.aspx?ParTree=151311&i={}".format(share_id))
            time.sleep(2)
            item = driver.find_element_by_class_name('torquoise')
        except:
            print("Page can not be loaded!")
            pass
    
    item.click()
    item = None

    while item is None:
        try:
            item = driver.find_element_by_class_name("dhx_not_active")
        except:
            print("Page can not be loaded!")
            pass
    
    content = driver.page_source 
    num_pages = content.count('onclick="this.grid.changePage(')

    if driver.find_element_by_class_name("dhx_not_active"):
        num_pages += 1

    pages_list = np.arange(1, num_pages+1, 1) 
    next_page = True

    while next_page:

        for page in pages_list:
            
            try:
                driver.find_element_by_link_text(str(page)).click()
            except:
                print("Page {} not found!".format(page))
            
            time.sleep(0.2)

            content = driver.page_source
            one_page = get_history(content)
            

        try:
            driver.find_element_by_link_text('>').click()
            time.sleep(0.2)
            content = driver.page_source 
            num_pages_tmp = content.count('onclick="this.grid.changePage(')

            if driver.find_element_by_class_name("dhx_not_active"):
                num_pages_tmp += 1

            pages_list = np.arange(num_pages+1, num_pages + num_pages_tmp+1, 1) 
            num_pages = num_pages + num_pages_tmp

        except:
            print("Element '>' doesn't exist!")
            next_page = False

    driver.quit()



get_all_history("67126881188552864", "/Users/USMAC/Desktop/chromedriver")

In [ ]:
def get_statements(share_id, chrome_driver):
    driver = webdriver.Chrome(chrome_driver)

    item = None
    while item is None:
        try:
            driver.get("http://www.tsetmc.com/loader.aspx?ParTree=151311&i={}".format(share_id))
            item = driver.page_source
        except:
            pass

    item = None
    while item is None:
        try:
            item = driver.find_element_by_class_name('lime')
        except:
            pass
    
    item.click()
    statements = []
    while len(statements) == 0:
        try:
            content = driver.page_source
            soup = BeautifulSoup(content)
            statements = soup.findAll('div', attrs={'class':'box1 red tbl z4_4'})
        except:
            pass
    
    df_list = []
    for statement in statements:

        table = statement

        header = []  

        for i, header_item in enumerate(table.find('tbody').findAll("tr")[0]):
            if i > 3:
                header.append(header_item.text + "_" + list(table.find('tbody').findAll("tr")[2].stripped_strings)[1])
            else:
                header.append(header_item.text + "_" + list(table.find('tbody').findAll("tr")[2].stripped_strings)[0])

        num_rows = len(table.find('tbody').findAll("tr")) - 3
        list_of_dicts = []

        for row in range(num_rows):
            the_row = []
            for tr in table.find('tbody').findAll("tr")[row + 3]:
                if tr.text == "":
                    the_row.append("    ")
                else:
                    the_row.append(tr.text)
            od = co.OrderedDict(zip(header,the_row))
            list_of_dicts.append(od)

        df = pd.DataFrame(list_of_dicts)
        df_list.append(df)
    
    driver.quit()
    return df_list

mylist = get_statements("67126881188552864", "/Users/USMAC/Desktop/chromedriver")